In [ ]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import os

In [ ]:
batch_size = 128
num_classes = 10
epochs = 75

In [ ]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
# Define student model
from keras.initializers import lecun_normal

def build_student():
    initializer = lecun_normal(seed=1)
    
    student = Sequential()
    student.add(Conv2D(16, (3, 3),
                     input_shape=x_train.shape[1:],
                      kernel_initializer=initializer))
    student.add(Activation('relu'))
    student.add(MaxPooling2D(pool_size=(2, 2)))

    student.add(Conv2D(32, (3, 3),
                       kernel_initializer=initializer))
    student.add(Activation('relu'))
    student.add(MaxPooling2D(pool_size=(2, 2)))

    student.add(Flatten())
    student.add(Dense(64, kernel_initializer=initializer))
    student.add(Activation('relu'))
    student.add(Dense(num_classes, kernel_initializer=initializer))
    student.add(Activation('softmax'))
    
    return student

In [ ]:
# Compile and prep data

student = build_student()

student.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

# Normalize pixel values
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Train

from keras.callbacks import EarlyStopping

history = student.fit(x_train, 
                      y_train,
                      epochs=epochs,
                      batch_size=batch_size,
                      validation_data=(x_test, y_test),
                      callbacks=[
                            EarlyStopping(monitor='val_loss', patience=5, min_delta=0.005)
                      ])

In [ ]:
# Plot loss

import matplotlib.pyplot as plt

plt.plot(range(1, len(history.history['val_loss'])+1), history.history['val_loss'], label='val')
plt.plot(range(1, len(history.history['loss'])+1), history.history['loss'], label='training')
plt.title('Progression of loss on benchmark')
plt.xlabel('epoch');
plt.ylabel('hard logloss');
plt.legend()
plt.savefig('benchmark_logloss.png')

In [ ]:
# Get overall acc
results = student.evaluate(x_test, y_test)

In [ ]:
student.summary()
print('Errors: ', len(x_test) - results[1] * len(x_test))
print('Accuracy: ', results[1]*100, '%')

In [ ]:
student.save('models/cifar10_student.h5')